# Big Red Soundscape
Notebook Playground 2  
_by Ahmed Sultan and Carly Hu, for Cornell Data Journal_

In [1]:
import json
import math

import numpy as np
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# update access keys here for Spotify API here :) 
client_id = 'f7661e34c0924317a8b928c0f977e6a7'
client_secret = 'd068ec3f0a98490e9c32be820916e934'

# sp is how we interface with the Spotify API
# read the docs here: https://spotipy.readthedocs.io/en/2.22.1/
sp = spotipy.Spotify(client_credentials_manager=\
                     SpotifyClientCredentials(client_id, client_secret))

<h3> Look at your own stats! </h3>

Feel free to put in your own spotify username in `selected_user` below, and if you run every cell, by the end of the notebook you will be able to see the information & statistics for __all of the songs that are on your public playlists__ (with duplicates removed). Have fun!

In [18]:
# Big Data, Machine Learning...
# https://www.youtube.com/watch?v=bqqCTC9nQDY

# let's aggregate an entire user's public library of songs
playlists = []
LIMIT = 20
selected_user = 'kaybae.'

query = sp.user_playlists(selected_user, limit=LIMIT)

n = 0

while len(query['items']) != 0:
    n += LIMIT
    for i in query['items']:
        playlists.append({'title': i['name'],'id': i['id'], 'length': i['tracks']['total']})
    if LIMIT == len(query['items']):
        query = sp.user_playlists(selected_user, limit=LIMIT, offset=n)
    else:
        break
        
# there's definitely a cleaner way to do this that we'll figure out
# im definitely lazy

In [19]:
playlists

[{'title': 'name 4 brothers or gtfo‼️',
  'id': '2zl5Uv8NGl6Q3g5dc6h6ov',
  'length': 255},
 {'title': 'tb to the good old days',
  'id': '4UlstL86DQfMRsS7GHIki0',
  'length': 458},
 {'title': 'sand in ma cheeks', 'id': '6rEePsA1H2OgSfFoDmTOnT', 'length': 427},
 {'title': 'top down w nowhere to go ',
  'id': '3xjLof9WnGgR1OpSUBnXiw',
  'length': 186},
 {'title': 'mindfuck me harder', 'id': '7uw8pNEERGXrt5Vw94mzsJ', 'length': 91},
 {'title': 'i just wanna feel smth',
  'id': '0GCkzwiNzJlh1lH2sR30In',
  'length': 231},
 {'title': 'with u', 'id': '51KgyolANfrXTVsSSjuk0D', 'length': 298},
 {'title': 'broc', 'id': '7s314m4v9udXpzfM2uILUP', 'length': 115},
 {'title': 'screaming', 'id': '52h6DmzwNslFlYAKxUo7cA', 'length': 63},
 {'title': 'sometimes there is no why',
  'id': '4gVaC4rRqXAB1NTkx7GHTu',
  'length': 426},
 {'title': 'more juice pls', 'id': '1ICgNreX5Eev9bPJUFu1LI', 'length': 703},
 {'title': 'sweatbox', 'id': '0PsINHKMfS0hH0b5vjSOZW', 'length': 79},
 {'title': 'violent vibrations 

In [17]:
# number of public playlists
print(len(playlists))

5


In [6]:
# making sure we can get all songs in a playlist rather than just the first 100
# https://stackoverflow.com/questions/55690063/is-there-a-method-to-retrieve-all-tracks-from-a-playlist-using-spotipy 

def user_playlist_tracks_full(user, playlist_id=None, fields=None):
    """ Get full details of the tracks of a playlist owned by a user.
        https://developer.spotify.com/documentation/web-api/reference/playlists/get-playlists-tracks/

        Parameters:
            - user - the id of the user
            - playlist_id - the id of the playlist
            - fields - which fields to return
    """

    # first run through also retrieves total no of songs in library
    response = sp.user_playlist_tracks(user, playlist_id, fields=None, limit=100)
    results = response["items"]

    # subsequently runs until it hits the user-defined limit or has read all songs in the library
    while len(results) < response["total"]:
        response = sp.user_playlist_tracks(
            user, playlist_id, fields=fields, limit=100, offset=len(results)
        )
        results.extend(response["items"])
    
    return results
        

In [7]:
# now, let's go through each playlist
# this will basically be the same as before

songs = []

for p in playlists:
    
    results = sp.playlist(p['id'])
    
    extended_results = user_playlist_tracks_full(user = selected_user, playlist_id = results['id']) # more than 100
    
    for r in extended_results:
        songs.append(r['track']['id']) 

print(len(songs)) # this is the total number of songs on your public playlists

2554


In [8]:
# helper function
def get_tracks_full(songs):
    results = []
    while len(results) < len(songs):
        response = (sp.tracks(songs[len(results):len(results) + 50]))['tracks']
        results.extend(response)
    return results

In [9]:
# this cell is creating a dataframe of your song information: artist, album, popularity, etc.

songs_meta = get_tracks_full(songs)

song_info = {'id': [], 'title': [], 'artist': [], 'album': [], 
             'explicit': [], 'popularity': []}
    
for i in songs_meta:
    # append id to id column
    song_info['id'].append(i['id'])
    
    # append song title to title column
    song_info['title'].append(i['name'])
    
    # append artists to artist column
    s = ', '
    artists = s.join([name['name'] for name \
                     in i['artists']])
    song_info['artist'].append(artists)
    
    # append album title to album column
    song_info['album'].append(i['album']['name'])
    
    # append explicit info to explicit column
    song_info['explicit'].append(i['explicit'])
    
    # append popularity info to popularity column
    song_info['popularity'].append(i['popularity'])
    
# convert song_info dictionary -> Pandas DataFrame
song_info_df = pd.DataFrame.from_dict(song_info)

song_info_df = song_info_df.drop_duplicates(subset=['id'])
song_info_df = song_info_df.dropna()
print(song_info_df.columns)
print(song_info_df.shape)

Index(['id', 'title', 'artist', 'album', 'explicit', 'popularity'], dtype='object')
(1125, 6)


In [10]:
#helper function 

def get_audio_features_full(songs):
    results = []
    while len(results) < len(songs):
        response = sp.audio_features(songs[len(results):len(results) + 100])
        results.extend(response)
    return results

In [11]:
# this cell is creating a dataframe of your song features: danceability, energy, loudness, etc

features_df = pd.DataFrame.from_dict(get_audio_features_full(songs))
features_df = features_df.drop_duplicates(subset=['id'])
features_df = features_df.dropna()
print(features_df.columns)
print(features_df.shape)

Index(['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
       'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms',
       'time_signature'],
      dtype='object')
(1125, 18)


In [12]:
# combining the two cleaned dataframes....
df = song_info_df.merge(features_df)
print(df.shape)
df

(1125, 23)


,id,title,artist,album,explicit,popularity,danceability,energy,key,loudness,...,instrumentalness,liveness,valence,tempo,type,uri,track_href,analysis_url,duration_ms,time_signature
0,3Kw7zkALCVxY4wmlnh2IWC,Cupid - Twin Ver.,FIFTY FIFTY,The Beginning: Cupid,False,82,0.782,0.587,11,-8.300,...,0.000002,0.3370,0.760,120.041,audio_features,spotify:track:3Kw7zkALCVxY4wmlnh2IWC,https://api.spotify.com/v1/tracks/3Kw7zkALCVxY...,https://api.spotify.com/v1/audio-analysis/3Kw7...,174253,4
1,3v5o91PrUtf0nmO6j8J7dZ,LEFT RIGHT,XG,SHOOTING STAR,False,83,0.767,0.721,1,-5.586,...,0.000002,0.1080,0.884,150.065,audio_features,spotify:track:3v5o91PrUtf0nmO6j8J7dZ,https://api.spotify.com/v1/tracks/3v5o91PrUtf0...,https://api.spotify.com/v1/audio-analysis/3v5o...,208066,4
2,1qrpoAMXodY6895hGKoUpA,You Belong With Me (Taylor’s Version),Taylor Swift,Fearless (Taylor's Version),False,84,0.632,0.773,6,-4.856,...,0.000000,0.0885,0.474,130.033,audio_features,spotify:track:1qrpoAMXodY6895hGKoUpA,https://api.spotify.com/v1/tracks/1qrpoAMXodY6...,https://api.spotify.com/v1/audio-analysis/1qrp...,231124,4
3,6YvqWjhGD8mB5QXcbcUKtx,Love Story (Taylor’s Version),Taylor Swift,Fearless (Taylor's Version),False,80,0.627,0.792,2,-4.311,...,0.000004,0.0845,0.415,119.054,audio_features,spotify:track:6YvqWjhGD8mB5QXcbcUKtx,https://api.spotify.com/v1/tracks/6YvqWjhGD8mB...,https://api.spotify.com/v1/audio-analysis/6Yvq...,235767,4
4,60wwxj6Dd9NJlirf84wr2c,Clarity,"Zedd, Foxes",Clarity,False,77,0.509,0.781,8,-3.480,...,0.000000,0.0749,0.176,128.000,audio_features,spotify:track:60wwxj6Dd9NJlirf84wr2c,https://api.spotify.com/v1/tracks/60wwxj6Dd9NJ...,https://api.spotify.com/v1/audio-analysis/60ww...,271427,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,4ER58qECydWokIsgqtysWu,딱 좋아(Just right),GOT7,Just right,False,62,0.760,0.781,11,-2.881,...,0.000000,0.1060,0.736,95.970,audio_features,spotify:track:4ER58qECydWokIsgqtysWu,https://api.spotify.com/v1/tracks/4ER58qECydWo...,https://api.spotify.com/v1/audio-analysis/4ER5...,222399,4
1121,3TDUktzfYXMWjkWqwoT5F1,Lights,BTS,MAP OF THE SOUL : 7 ~ THE JOURNEY ~,False,62,0.608,0.644,5,-5.588,...,0.000000,0.1240,0.431,99.005,audio_features,spotify:track:3TDUktzfYXMWjkWqwoT5F1,https://api.spotify.com/v1/tracks/3TDUktzfYXMW...,https://api.spotify.com/v1/audio-analysis/3TDU...,292853,4
1122,645YBsxhFPSOKxGep3uHLv,Love Maze,BTS,Love Yourself 轉 'Tear',False,0,0.636,0.876,9,-4.652,...,0.000000,0.0728,0.759,149.932,audio_features,spotify:track:645YBsxhFPSOKxGep3uHLv,https://api.spotify.com/v1/tracks/645YBsxhFPSO...,https://api.spotify.com/v1/audio-analysis/645Y...,221529,4
1123,1ZPpGPko8oULTmi34rozMh,Trivia 轉 : Seesaw,BTS,Love Yourself 結 'Answer',False,0,0.707,0.791,5,-4.481,...,0.000000,0.1020,0.678,106.985,audio_features,spotify:track:1ZPpGPko8oULTmi34rozMh,https://api.spotify.com/v1/tracks/1ZPpGPko8oUL...,https://api.spotify.com/v1/audio-analysis/1ZPp...,246334,4


In [14]:
features = ['popularity','danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
       'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']

feature_averages = []
for feature in features:
    avg =  df[feature].mean()
    feature_averages.append(avg)

print(feature_averages)

[59.57866666666666, 0.6631448888888883, 0.6532432888888893, 5.207111111111111, -6.175544888888897, 0.6382222222222222, 0.09992711111111127, 0.24010029211555528, 0.012390474968888903, 0.17252026666666648, 0.5153368888888885, 120.90672177777776]
